In [1]:
# Setup the Jupyter version of Dash
from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import datetime
from PIL import Image


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import CRUD functionality
import CRUD



###########################
# Data Manipulation / Model
###########################

# Instantiate CRUD object with username and password values
username = "aacuser"
password = "SNHU1234"
shelter = CRUD.AnimalShelter # CRUD object
shelter.__init__(CRUD.AnimalShelter, username, password) # initialize


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read(shelter, {}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

# Import Company Logo
pil_image = Image.open('Assets/Grazioso-Salvare-Logo.png')




#########################
# Dashboard Layout / View
#########################
app = JupyterDash(prevent_initial_callbacks = True)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(
        # This contains the hyper-linked logo image
        # and the title to the dashboard
        html.Div([
            html.A(
                href = 'https://www.snhu.edu',
                children = [
                    html.Img(
                        alt = "Link to client's home page",
                        src=pil_image,
                        style = {'width': '200px', 'height': '200px'}
                    )
                ]
            ),
            html.B(html.H1('AAC Shelter Dashboard',
                          style = {'color': '#c9134b'})) # same color as logo
        ])
    ),
    html.Br(),
    html.Hr(),
    # This contains a selectable radio to filter the dataset
    dcc.RadioItems(id = 'rescue-type-radio',
                   options = [
                       {'label': 'Water Rescue', 'value': 1},
                       {'label': 'Mountain Rescue', 'value': 2},
                       {'label': 'Disaster Rescue', 'value': 3},
                       {'label': 'Reset', 'value': 4}
                   ],
                   value = 4,
                   inline = True
                  ),
    html.Br(),
    # This contains the interactible dataset
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # These features make the interactive data table user-friendly for the client
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "single",
        row_deletable = False,
        selected_columns = [],
        selected_rows = [0],
        page_action = "native",
        page_current = 0,
        page_size = 10
    ),
    html.Br(),
    html.Hr(),
    html.Br(),
    # This contains the geolocation map, and the pie graph of breeds
    html.Div(
        [
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'float': 'right', 'margin': '10px', 'padding': '10px'}
            ),
            dcc.Graph(
                id='breed-graph',
                style={'float': 'right', 'margin': '10px', 'padding': '10px'}
            )
        ], style={'margin': '10px', 'padding': '10px'}
    ),
    html.Br(),
    # Unique identifier code
    html.Header("Morgan Getkin SNHU CS-340"),
    html.Br()
])




#############################################
# Interaction Between Components / Controller
#############################################

# This callback will allow radio buttons to show special query results
@app.callback(
    Output('datatable-id', 'data'),
    Input('rescue-type-radio', 'value')
)
def on_selection(value, **kwargs):
    if value == 1:
        df = pd.DataFrame.from_records(shelter.read(shelter, {"$or": [
            {"breed" : "Labrador Retriever Mix"},
            {"breed" : "Chesapeake Bay Retriever"},
            {"breed" : "Newfoundland"}
            ], "sex_upon_outcome" : "Intact Female",
            "$and": [{"age_upon_outcome_in_weeks": {"$lte": 156}},
                   {"age_upon_outcome_in_weeks": {"$gte": 26}}]
        }))
    elif value == 2:
        df = pd.DataFrame.from_records(shelter.read(shelter, {"$or": [
            {"breed" : "German Shepherd"},
            {"breed" : "Alaskan Malamute"},
            {"breed" : "Old English Sheepdog"},
            {"breed" : "Siberian Husky"},
            {"breed" : "Rottweiler"}
            ], "sex_upon_outcome" : "Intact Male",
            "$and": [{"age_upon_outcome_in_weeks": {"$lte": 156}},
                   {"age_upon_outcome_in_weeks": {"$gte": 26}}]
        }))
    elif value == 3:
        df = pd.DataFrame.from_records(shelter.read(shelter, {"$or": [
            {"breed" : "Doberman Pinscher"},
            {"breed" : "German Shepherd"},
            {"breed" : "Golden Retriever"},
            {"breed" : "Bloodhound"},
            {"breed" : "Rottweiler"}
            ], "sex_upon_outcome" : "Intact Male",
            "$and": [{"age_upon_outcome_in_weeks": {"$lte": 300}},
                   {"age_upon_outcome_in_weeks": {"$gte": 20}}]
        }))
    else:
        df = pd.DataFrame.from_records(shelter.read(shelter, {}))
    
    # Cleanup Mongo _id field
    df.drop(columns = ['_id'], inplace = True)
    return df.to_dict('records')


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns, **kwargs):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the pie chart for the selected data entry
@app.callback(
    Output('breed-graph', 'figure'),
    Input('datatable-id', 'derived_virtual_data')
)
def generate_chart(viewData, **kwargs):
    dff = pd.DataFrame.from_dict(viewData)
    # Group by breed field and count instances of each breed
    dff = dff.groupby(['breed'])['breed'].count().reset_index(name = 'count')
    df_draw = dff.copy()
    # There are many different breeds listed, therefore the raw-data
    # pie chart is almost unreadable. Most of the pie is comprised of
    # tiny slices (<1%). To group these all into an unspecified "other"
    # slice would cause most of the chart to be this "other" category,
    # and therefore non-descriptive, defeating the purpose of the chart
    """
    # Group sub-10 counts into 'other' category
    df_draw.loc[df_draw['count'] <= 100, 'breed'] = 'Other'
    
    #df_draw = df.groupby(['breed'])['count'].sum().reset_index()
    #print(df_draw)
    """
    
    # plot the graph
    fig = px.pie(df_draw, values = 'count', names = 'breed', title = 'Breeds', hole = 0.3)
    return fig


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index, **kwargs):
    # Code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]
    
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(
            style = {'width': '1000px', 'height': '500px'},
            center = [30.75, -97.48], 
            zoom = 10, 
            children = [
                dl.TileLayer(id = "base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(
                    position = [
                        dff.iloc[row, 13],
                        dff.iloc[row, 14]
                    ],
                    children = [
                        dl.Tooltip(dff.iloc[row, 9]),
                        dl.Popup([
                            html.H1(dff.iloc[row, 9]),
                            html.P(dff.iloc[row, 4])
                        ])
                    ]
                )
            ]
        )
    ]
    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:25479/
